In [1]:
! pip install pycocotools
! pip install -U ultralytics

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 54.6 MB/s  0:00:00

  Attempting uninstall: ultralytics-thop

    Found existing installation: ultralytics-thop 2.0.17

    Uninstalling ultralytics-thop-2.0.17:

      Successfully uninstalled ultralytics-thop-2.0.17

  Attempting uninstall: ultralytics

    Found existing installation: ultralytics 8.3.221

    Uninstalling ultralytics-8.3.221:

      Successfully uninstalled ultralytics-8.3.221

   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   ---------------------------------------- 2/2 [ultralytics]



In [51]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import kagglehub
import torchvision
import pandas as pd
import torchvision.tv_tensors as tv_tensors


from tqdm import tqdm
from torch.utils.data import DataLoader, WeightedRandomSampler
from PIL import Image, ImageEnhance, ImageFilter
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from torchvision.transforms import v2
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torchvision.models.detection.ssd import SSD300_VGG16_Weights

In [52]:
# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.mps.is_available() else device)
print(f"Using device: {device}")

Using device: cuda


## 하이퍼 파라미터 조정

In [53]:
batch_sizes = 16
epoch = 100

# 1차 데이터

## 데이터 로드

In [ ]:
path = r"C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001"
train_dir = os.path.join(path, "images")
annotation_dir = r"C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\final_annotations.json"



In [55]:
img_files = [f for f in os.listdir(train_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"Train 수: {len(img_files)}")



Train 수: 3176


## 함수 생성

In [ ]:
import torchvision.tv_tensors as tv_tensors

class CustomDataset(Dataset):
    def __init__(self, image_dir, annotation_mapping, image_list, transforms=None):
        self.image_dir = image_dir
        self.annotation_mapping = annotation_mapping
        self.image_list = image_list
        self.transforms = transforms

    # ✅ 빠진 부분 추가!
    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        name = self.image_list[idx]
        image_path = os.path.join(self.image_dir, name)
        image = Image.open(image_path).convert("RGB")

        anns = self.annotation_mapping.get(name, [])
        boxes = [ann["bbox"] for ann in anns]
        labels = [ann["category_id"] for ann in anns]


        # ✅ BoundingBoxes 타입 변환 (v2용)
        image = tv_tensors.Image(image)
        boxes = tv_tensors.BoundingBoxes(
            boxes,
            format=tv_tensors.BoundingBoxFormat.XYXY,
            canvas_size=image.shape[-2:]
        )
        labels = torch.as_tensor(labels, dtype=torch.int64)

        # ✅ v2 transform 적용
        if self.transforms:
            image, boxes = self.transforms(image, boxes)

        # ✅ Normalize는 bbox 이후에만 적용
        image = self.normalize(image)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": idx
        }

        return image, target


In [39]:
from torch.utils.data import Dataset
from PIL import Image
import os

class TestDataset(Dataset):
    def __init__(self, image_dir, image_list, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms
        self.image_files = image_list  # 테스트 이미지 리스트 (확장자 없음)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # 이미지 파일 경로
        image_file = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_file)

        # 이미지 로드
        image = Image.open(image_path).convert("RGB")

        # Transform 적용
        if self.transforms:
            image = self.transforms(image)

        return image, self.image_files[idx]  # 이미지와 파일 이름 반환


In [56]:
import os
import json
import hashlib

def extract_classes_from_json_groups(json_groups):
    category_map = {}
    seen_hashes = set()

    for paths in json_groups.values():
        for path in paths:
            with open(path, 'r') as f:
                data = json.load(f)
                categories = data.get("categories", [])

                # 중복 방지: 같은 구조면 스킵
                hash_key = hashlib.md5(json.dumps(categories, sort_keys=True).encode()).hexdigest()
                if hash_key in seen_hashes:
                    continue
                seen_hashes.add(hash_key)

                for cat in categories:
                    category_map[cat["id"]] = cat["name"]


    sorted_categories = sorted(category_map.items())
    classes = ["__background__"] + [name for _, name in sorted_categories]

    return classes

In [82]:
import json
from collections import defaultdict

# JSON 로드
with open(annotation_dir, "r", encoding="utf-8") as f:
    coco_data = json.load(f)

# ✅ 이미지별 어노테이션 매핑 생성
annotation_mapping = defaultdict(list)
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    image_name = next(
        (img["file_name"] for img in coco_data["images"] if img["id"] == image_id),
        None
    )
    if image_name:
        annotation_mapping[image_name].append(ann)

print("✅ annotation_mapping 생성 완료")
print("예시 키:", list(annotation_mapping.keys())[:5])
print("첫 번째 이미지의 어노테이션 수:", len(annotation_mapping[list(annotation_mapping.keys())[0]]))


✅ annotation_mapping 생성 완료
예시 키: ['K-001900-016548-018110-021026_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-021026_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-027926_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_90_000_200.png']
첫 번째 이미지의 어노테이션 수: 4


In [58]:
import json
with open(annotation_dir, "r", encoding="utf-8") as f:
    data = json.load(f)

cat_ids = sorted(set([ann["category_id"] for ann in data["annotations"]]))
print("카테고리 ID의 예시:", cat_ids[:50])
print("카테고리 개수:", len(cat_ids))

카테고리 ID의 예시: [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 19551, 19606, 19860, 20013, 20237, 20876, 21025, 21324, 21770, 22073, 22346, 22361, 22626, 23202, 23222, 24849, 25366, 25437, 25468, 27652, 27732]
카테고리 개수: 73


In [128]:
from sklearn.model_selection import train_test_split

# 전체 이미지 파일 이름 리스트
image_names = list(annotation_mapping.keys())

# 이미지 리스트를 train/val로 나누기
train_list, val_list = train_test_split(image_names, test_size=0.2, random_state=42)

# 어노테이션 매핑도 함께 나누기
annotation_mapping_train = {name: annotation_mapping[name] for name in train_list}
annotation_mapping_val   = {name: annotation_mapping[name] for name in val_list}

print("✅ train/val 분리 완료")
print(f"Train 이미지 수: {len(train_list)}")
print(f"Val 이미지 수: {len(val_list)}")


✅ train/val 분리 완료
Train 이미지 수: 3052
Val 이미지 수: 763


In [60]:
from torchvision.transforms import v2

train_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((640, 640)),
])

valid_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((640, 640)),
])

In [62]:
from torchvision.datasets import CocoDetection
from torchvision import transforms

train_dataset = CustomDataset(
    image_dir=train_dir,
    annotation_mapping=annotation_mapping_train,
    image_list=train_list,
    transforms=train_transform
)

valid_dataset = CustomDataset(
    image_dir=train_dir,  # validation 이미지도 동일 폴더 내에 있는 경우
    annotation_mapping=annotation_mapping_val,
    image_list=val_list,
    transforms=valid_transform
)


In [63]:
def collate_fn(batch):
    return tuple(zip(*batch))


In [64]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size = batch_sizes, shuffle=True, collate_fn=collate_fn)

valid_loader = DataLoader(valid_dataset, batch_size = batch_sizes, shuffle=False, collate_fn=collate_fn)


# 데이터 크기 출력
print(f"Train 데이터셋 크기: {len(train_dataset)}")
print(f"Validation 데이터셋 크기: {len(valid_dataset)}")

Train 데이터셋 크기: 3052
Validation 데이터셋 크기: 763


## YOLO 변환 코드

In [65]:
# ============================================
# ✅ COCO → YOLO 변환 파트 (기존 데이터 로딩 이후 추가)
# ============================================

import os, json
from tqdm import tqdm

# 기존 path와 train_dir 그대로 사용
train_json = os.path.join(path, "merged_coco_final_train.json")
val_json   = os.path.join(path, "merged_coco_final_valid.json")

# ✅ (1) train/val JSON 파일 저장
# 기존 annotation_mapping_train / val 기반으로 저장
def save_coco_split_json(mapping, split_name, save_path):
    images = []
    annotations = []
    categories = coco_data["categories"]

    img_id_map = {}
    for i, name in enumerate(mapping.keys()):
        img_info = next((img for img in coco_data["images"] if img["file_name"] == name), None)
        if img_info:
            new_img = img_info.copy()
            new_img["id"] = i + 1
            img_id_map[img_info["id"]] = new_img["id"]
            images.append(new_img)

            for ann in mapping[name]:
                new_ann = ann.copy()
                new_ann["image_id"] = new_img["id"]
                annotations.append(new_ann)

    coco_split = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(coco_split, f, ensure_ascii=False, indent=2)
    print(f"✅ {split_name} JSON 저장 완료 → {save_path}")


save_coco_split_json(annotation_mapping_train, "train", train_json)
save_coco_split_json(annotation_mapping_val, "val", val_json)

# ✅ (2) YOLO 폴더 구조 생성
yolo_root = os.path.join(path, "yolo_dataset")
os.makedirs(os.path.join(yolo_root, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "labels/val"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)


# ✅ (3) COCO → YOLO 변환 함수 (수정된 버전)
def coco_to_yolo(json_path, split):
    with open(json_path, "r", encoding="utf-8") as f:
        coco = json.load(f)

    # ✅ 카테고리 매핑 테이블 생성 (category_id → YOLO 인덱스)
    category_id_to_index = {cat["id"]: i for i, cat in enumerate(coco["categories"])}
    print(f"📦 category_id → class_index 매핑 예시: {list(category_id_to_index.items())[:5]}")

    images = {img["id"]: img for img in coco["images"]}
    ann_by_img = {img_id: [] for img_id in images.keys()}

    for ann in coco["annotations"]:
        ann_by_img[ann["image_id"]].append(ann)

    for img_id, anns in tqdm(ann_by_img.items(), desc=f"Converting {split}"):
        img_info = images[img_id]
        w, h = img_info["width"], img_info["height"]
        file_name = img_info["file_name"]
        file_stem = os.path.splitext(file_name)[0]
        label_path = os.path.join(yolo_root, f"labels/{split}/{file_stem}.txt")

        lines = []
        for ann in anns:
            # ✅ COCO: [x_min, y_min, width, height]
            x_min, y_min, box_w, box_h = ann["bbox"]

            x_center = (x_min + box_w / 2) / w
            y_center = (y_min + box_h / 2) / h
            box_w /= w
            box_h /= h

            # ✅ 잘못된 category_id 보정
            raw_id = ann["category_id"]
            cls_id = category_id_to_index.get(raw_id, 0)  # 존재하지 않으면 0으로 대체

            lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}")

        with open(label_path, "w", encoding="utf-8") as lf:
            lf.write("\n".join(lines))

    print(f"✅ {split} 변환 완료 → {len(images)}개 이미지")


# ✅ (4) 변환 실행
coco_to_yolo(train_json, "train")
coco_to_yolo(val_json, "val")


# ✅ (5) data.yaml 생성
yaml_path = os.path.join(yolo_root, "data.yaml")
categories = json.load(open(train_json))["categories"]

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"train: {yolo_root}/images/train\n")
    f.write(f"val: {yolo_root}/images/val\n")
    f.write(f"nc: {len(categories)}\n")
    f.write("names:\n")
    for i, c in enumerate(categories):
        f.write(f"  {i}: {c['name']}\n")

print(f"🎯 data.yaml 생성 완료 → {yaml_path}")


✅ train JSON 저장 완료 → C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\merged_coco_final_train.json
✅ val JSON 저장 완료 → C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\merged_coco_final_valid.json
📦 category_id → class_index 매핑 예시: [(16547, 0), (32309, 1), (33879, 2), (3482, 3), (31704, 4)]


Converting train: 100%|██████████| 3052/3052 [00:00<00:00, 3629.94it/s]


✅ train 변환 완료 → 3052개 이미지
📦 category_id → class_index 매핑 예시: [(16547, 0), (32309, 1), (33879, 2), (3482, 3), (31704, 4)]


Converting val: 100%|██████████| 763/763 [00:00<00:00, 3779.35it/s]


✅ val 변환 완료 → 763개 이미지
🎯 data.yaml 생성 완료 → C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml


In [69]:
import pandas as pd
import json
import os
from ultralytics import YOLO

model_size = "s"
model_path = f"yolov8{model_size}.pt"
run_name = f"predict_{model_size}"
project_dir = f"C:/content/runs/detect"

# ✅ 모델 로드 및 학습
model = YOLO(model_path)
results = model.train(
    data=r"C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml",
    epochs=100,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    save=True,
    save_txt=True,
    save_conf=True,
    verbose=True,
    project=project_dir,
    name=run_name,   # ⭐ 사이즈에 따라 결과 폴더 분리됨
    exist_ok=True
)

# ✅ 학습 완료 후 CSV → JSON 저장
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df_selected = df[["      box_loss", "   cls_loss", "   dfl_loss", "     mAP50", "  mAP50-95"]]
    df_selected.columns = [col.strip() for col in df_selected.columns]  # 열 이름 정리
    df_selected.to_json(json_path, orient="records", indent=2)
    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_s, nbs=64, nms=False, opset=None, optimize=False, optimi

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all       1154       2565      0.555      0.997      0.603      0.601
            100mg         47         47      0.573          1       0.57       0.57
                      41         41      0.603          1      0.636      0.635
        ()         28         28      0.444          1      0.442      0.441
    ()()         42         42      0.583          1        0.7      0.693
          500/20mg         28         28      0.528          1      0.549      0.547
              5mg         38         38      0.594          1      0.709      0.709
               10mg         29         29      0.509          1      0.559      0.559
          ()         40         40      0.548          1      0.561      0.559
          10mg/         42         42      0.627          1      0.624      0.616
          800mg         25         25      0.439          1      0.436       0.43
             20mg         23         23      0.426          1      0.447      0.447
         30mg

KeyError: "None of [Index(['      box_loss', '   cls_loss', '   dfl_loss', '     mAP50',\n       '  mAP50-95'],\n      dtype='object')] are in the [columns]"

In [ ]:
import pandas as pd
import json
import os
import time

# 모델 사이즈 설정 (예: "s", "n", "m", "l")
model_size = "s"  # 원하는 사이즈로 변경
run_name = f"predict_{model_size}"
project_dir = "C:/content/runs/detect"

# 경로 설정
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

# 파일 존재 여부 확인 후 처리
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)

    # 열 이름 공백 제거
    df.columns = [col.strip() for col in df.columns]

    # 필요한 열만 추출 (존재하는 것만)
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    existing_keys = [key for key in keys if key in df.columns]
    df_selected = df[existing_keys]

    # 학습 시간은 없으므로 현재 시점에서는 생략하거나 "N/A" 처리
    output = {
        "model": f"yolov8{model_size}.pt",
        "epochs": len(df),
        "elapsed_minutes": "N/A",
        "logs": df_selected.to_dict(orient="records")
    }

    # 저장
    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)

    print(f"✅ 이미 학습된 로그로부터 JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일이 존재하지 않습니다.")


✅ 이미 학습된 로그로부터 JSON 저장 완료: C:/content/runs/detect\predict_s\results.json


In [ ]:
import os
import json
import re

# 경로 설정
project_dir = "C:/content/runs/detect"
run_name = "predict_s"  # run_name은 실험명에 맞게 수정
log_path = os.path.join(project_dir, run_name, "train.log")
json_output_path = os.path.join(project_dir, run_name, "epoch_logs.json")

# 로그 파싱
logs = []
with open(log_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# 헤더 찾기
headers = []
for line in lines:
    if re.search(r'\bEpoch\b', line):
        headers = re.findall(r'\S+', line)
        break

# 데이터 파싱
for line in lines:
    if re.match(r'^\d+/\d+', line.strip()):
        values = re.findall(r'\S+', line)
        log = dict(zip(headers, values))
        logs.append(log)

# 저장할 JSON 구조
output = {
    "run_name": run_name,
    "total_epochs": len(logs),
    "logs": logs
}

# JSON 저장
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2)

print(f"✅ 에폭별 로그 JSON 저장 완료: {json_output_path}")


Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_n, nbs=64, nms=False, opset=None, optimize=False, optimi

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all       1154       2565      0.546      0.976      0.608      0.605
            100mg         47         47      0.416          1      0.645      0.637
                      41         41      0.603          1       0.67       0.67
        ()         28         28      0.439          1      0.593      0.591
    ()()         42         42      0.583          1      0.629      0.627
          500/20mg         28         28      0.528          1      0.606      0.606
              5mg         38         38      0.594          1      0.623      0.623
               10mg         29         29      0.509          1      0.594      0.593
          ()         40         40      0.538          1       0.65      0.647
          10mg/         42         42      0.621          1      0.689       0.68
          800mg         25         25      0.445      0.995      0.503      0.498
             20mg         23         23      0.426          1      0.493       0.49
         30mg

In [75]:
import pandas as pd
import json
import os
import time
from ultralytics import YOLO

# 모델 설정
model_size = "m"
model_path = f"yolov8{model_size}.pt"
run_name = f"predict_{model_size}"
project_dir = "C:/content/runs/detect"

# 시간 측정 시작
start_time = time.time()

# 모델 학습
model = YOLO(model_path)
results = model.train(
    data=r"C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml",
    epochs=100,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    save=True,
    save_txt=True,
    save_conf=True,
    verbose=True,
    project=project_dir,
    name=run_name,
    exist_ok=True
)

# 시간 측정 종료
elapsed_minutes = round((time.time() - start_time) / 60, 2)

# 경로 설정
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

# CSV → JSON
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)

    # 열 이름 자동 정리
    df.columns = [col.strip() for col in df.columns]

    # 존재하는 열만 안전하게 추출
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    existing_keys = [key for key in keys if key in df.columns]
    df_selected = df[existing_keys]

    # JSON 저장 형식
    output = {
        "model": f"yolov8{model_size}.pt",
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_m, nbs=64, nms=False, opset=None, optimize=False, optimi

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all       1154       2565      0.557      0.996      0.615      0.614
            100mg         47         47      0.573          1      0.689      0.687
                      41         41      0.603          1      0.669      0.669
        ()         28         28      0.417      0.893      0.511      0.511
    ()()         42         42      0.583          1      0.627      0.618
          500/20mg         28         28      0.528          1      0.583       0.58
              5mg         38         38      0.594          1      0.607      0.607
               10mg         29         29      0.509          1      0.507      0.507
          ()         40         40      0.548          1       0.62       0.62
          10mg/         42         42      0.627          1      0.715       0.71
          800mg         25         25      0.439          1      0.456      0.448
             20mg         23         23      0.426          1      0.494      0.494
         30mg

In [ ]:
import pandas as pd
import json
import os
import time
from ultralytics import YOLO

# 모델 설정
model_size = "l"
model_path = f"yolov8{model_size}.pt"
run_name = f"predict_{model_size}"
project_dir = "C:/content/runs/detect"

# 시간 측정 시작
start_time = time.time()

# 모델 학습
model = YOLO(model_path)
results = model.train(
    data=r"C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml",
    epochs=100,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    save=True,
    save_txt=True,
    save_conf=True,
    verbose=True,
    project=project_dir,
    name=run_name,
    exist_ok=True
)

# 시간 측정 종료
elapsed_minutes = round((time.time() - start_time) / 60, 2)

# 경로 설정
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

# CSV → JSON
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)

    # 열 이름 자동 정리
    df.columns = [col.strip() for col in df.columns]

    # 존재하는 열만 안전하게 추출
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    existing_keys = [key for key in keys if key in df.columns]
    df_selected = df[existing_keys]

    # JSON 저장 형식
    output = {
        "model": f"yolov8{model_size}.pt",
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


# 2차 학습

In [138]:
path_2 = r"C:\Users\kdh12\python\dataset_v2"
train_dir_2 = os.path.join(path, "images")
annotation_dir_2 = r"C:\Users\kdh12\python\dataset_v2\merged_annotations.json"



In [139]:
img_files_2 = [f for f in os.listdir(train_dir_2) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"Train 수: {len(img_files_2)}")



Train 수: 3176


In [140]:
# JSON 로드
with open(annotation_dir_2, "r", encoding="utf-8") as f:
    coco_data = json.load(f)

# ✅ 이미지별 어노테이션 매핑 생성
annotation_mapping_2 = defaultdict(list)
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    image_name = next(
        (img["file_name"] for img in coco_data["images"] if img["id"] == image_id),
        None
    )
    if image_name:
        annotation_mapping_2[image_name].append(ann)

print("✅ annotation_mapping 생성 완료")
print("예시 키:", list(annotation_mapping_2.keys())[:5])
print("첫 번째 이미지의 어노테이션 수:", len(annotation_mapping_2[list(annotation_mapping_2.keys())[0]]))


✅ annotation_mapping 생성 완료
예시 키: ['K-001900-016548-018110-021026_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-021026_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-027926_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_90_000_200.png']
첫 번째 이미지의 어노테이션 수: 4


In [143]:

# 전체 이미지 파일 이름 리스트
image_names_2 = list(annotation_mapping_2.keys())



# 어노테이션 매핑도 함께 나누기
annotation_mapping_train = {name: annotation_mapping_2[name] for name in image_names_2 }
train_list_2  = {name: annotation_mapping_2[name] for name in image_names_2 }

print("✅ train/val 분리 완료")
print(f"Train 이미지 수: {len(train_list_2)}")
print(f"Val 이미지 수: {len(val_list)}")

✅ train/val 분리 완료
Train 이미지 수: 644
Val 이미지 수: 763


In [144]:
from torchvision.transforms import v2

train_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((640, 640)),
])

valid_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((640, 640)),
])

In [147]:
from torchvision.datasets import CocoDetection
from torchvision import transforms

train_dataset = CustomDataset(
    image_dir=train_dir,
    annotation_mapping=annotation_mapping_train,
    image_list=train_list_2,
    transforms=train_transform
)

valid_dataset = CustomDataset(
    image_dir=train_dir,  # validation 이미지도 동일 폴더 내에 있는 경우
    annotation_mapping=annotation_mapping_val,
    image_list=val_list,
    transforms=valid_transform
)


In [148]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [149]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size = batch_sizes, shuffle=True, collate_fn=collate_fn)

valid_loader = DataLoader(valid_dataset, batch_size = batch_sizes, shuffle=False, collate_fn=collate_fn)


# 데이터 크기 출력
print(f"Train 데이터셋 크기: {len(train_dataset)}")
print(f"Validation 데이터셋 크기: {len(valid_dataset)}")

Train 데이터셋 크기: 644
Validation 데이터셋 크기: 763


In [150]:
# ============================================
# ✅ COCO → YOLO 변환 파트 (기존 데이터 로딩 이후 추가)
# ============================================

import os, json
from tqdm import tqdm

# 기존 path와 train_dir 그대로 사용
train_json = os.path.join(path_2, "merged_coco_final_train.json")
val_json   = os.path.join(path_2, "merged_coco_final_valid.json")

# ✅ (1) train/val JSON 파일 저장
# 기존 annotation_mapping_train / val 기반으로 저장
def save_coco_split_json(mapping, split_name, save_path):
    images = []
    annotations = []
    categories = coco_data["categories"]

    img_id_map = {}
    for i, name in enumerate(mapping.keys()):
        img_info = next((img for img in coco_data["images"] if img["file_name"] == name), None)
        if img_info:
            new_img = img_info.copy()
            new_img["id"] = i + 1
            img_id_map[img_info["id"]] = new_img["id"]
            images.append(new_img)

            for ann in mapping[name]:
                new_ann = ann.copy()
                new_ann["image_id"] = new_img["id"]
                annotations.append(new_ann)

    coco_split = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(coco_split, f, ensure_ascii=False, indent=2)
    print(f"✅ {split_name} JSON 저장 완료 → {save_path}")


save_coco_split_json(annotation_mapping_train, "train", train_json)
save_coco_split_json(annotation_mapping_val, "val", val_json)

# ✅ (2) YOLO 폴더 구조 생성
yolo_root = os.path.join(path_2, "yolo_dataset")
os.makedirs(os.path.join(yolo_root, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "labels/val"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)


# ✅ (3) COCO → YOLO 변환 함수 (수정된 버전)
def coco_to_yolo(json_path, split):
    with open(json_path, "r", encoding="utf-8") as f:
        coco = json.load(f)

    # ✅ 카테고리 매핑 테이블 생성 (category_id → YOLO 인덱스)
    category_id_to_index = {cat["id"]: i for i, cat in enumerate(coco["categories"])}
    print(f"📦 category_id → class_index 매핑 예시: {list(category_id_to_index.items())[:5]}")

    images = {img["id"]: img for img in coco["images"]}
    ann_by_img = {img_id: [] for img_id in images.keys()}

    for ann in coco["annotations"]:
        ann_by_img[ann["image_id"]].append(ann)

    for img_id, anns in tqdm(ann_by_img.items(), desc=f"Converting {split}"):
        img_info = images[img_id]
        w, h = img_info["width"], img_info["height"]
        file_name = img_info["file_name"]
        file_stem = os.path.splitext(file_name)[0]
        label_path = os.path.join(yolo_root, f"labels/{split}/{file_stem}.txt")

        lines = []
        for ann in anns:
            # ✅ COCO: [x_min, y_min, width, height]
            x_min, y_min, box_w, box_h = ann["bbox"]

            x_center = (x_min + box_w / 2) / w
            y_center = (y_min + box_h / 2) / h
            box_w /= w
            box_h /= h

            # ✅ 잘못된 category_id 보정
            raw_id = ann["category_id"]
            cls_id = category_id_to_index.get(raw_id, 0)  # 존재하지 않으면 0으로 대체

            lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}")

        with open(label_path, "w", encoding="utf-8") as lf:
            lf.write("\n".join(lines))

    print(f"✅ {split} 변환 완료 → {len(images)}개 이미지")


# ✅ (4) 변환 실행
coco_to_yolo(train_json, "train")
coco_to_yolo(val_json, "val")


# ✅ (5) data.yaml 생성
yaml_path = os.path.join(yolo_root, "data.yaml")
categories = json.load(open(train_json))["categories"]

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"train: {yolo_root}/images/train\n")
    f.write(f"val: {yolo_root}/images/val\n")
    f.write(f"nc: {len(categories)}\n")
    f.write("names:\n")
    for i, c in enumerate(categories):
        f.write(f"  {i}: {c['name']}\n")

print(f"🎯 data.yaml 생성 완료 → {yaml_path}")


✅ train JSON 저장 완료 → C:\Users\kdh12\python\dataset_v2\merged_coco_final_train.json
✅ val JSON 저장 완료 → C:\Users\kdh12\python\dataset_v2\merged_coco_final_valid.json
📦 category_id → class_index 매핑 예시: [(16547, 0), (32309, 1), (33879, 2), (3482, 3), (31704, 4)]


Converting train: 100%|██████████| 644/644 [00:00<00:00, 4583.93it/s]


✅ train 변환 완료 → 644개 이미지
📦 category_id → class_index 매핑 예시: [(16547, 0), (32309, 1), (33879, 2), (3482, 3), (31704, 4)]


Converting val: 100%|██████████| 146/146 [00:00<00:00, 4067.33it/s]

✅ val 변환 완료 → 146개 이미지
🎯 data.yaml 생성 완료 → C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml


In [151]:
import os, shutil, glob, random

img_src  = os.path.join(path_2, "images")         # 원본 이미지 폴더
yolo_root = os.path.join(path_2, "yolo_dataset")

# 대상 폴더 생성
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)

# train/val 라벨에 해당하는 파일 목록 읽기
train_labels = glob.glob(os.path.join(yolo_root, "labels/train", "*.txt"))
val_labels   = glob.glob(os.path.join(yolo_root, "labels/val", "*.txt"))

# 각 라벨에 대응하는 이미지 파일 복사
def copy_images(label_paths, split):
    copied = 0
    for label_path in label_paths:
        file_stem = os.path.splitext(os.path.basename(label_path))[0]
        for ext in [".jpg", ".png", ".jpeg"]:
            img_path = os.path.join(img_src, file_stem + ext)
            if os.path.exists(img_path):
                dst_path = os.path.join(yolo_root, f"images/{split}", os.path.basename(img_path))
                shutil.copy2(img_path, dst_path)
                copied += 1
                break
    print(f"✅ {split} 이미지 복사 완료: {copied}개")

copy_images(train_labels, "train")
copy_images(val_labels, "val")


✅ train 이미지 복사 완료: 644개
✅ val 이미지 복사 완료: 253개


# 학습

## YOLO11m

In [153]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# 기존 모델 로드 (best.pt)
base_model_dir = "C:/content/runs/detect/predict_m/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 새 프로젝트 설정
model_size = "m"
run_name = f"predict_{model_size}_fine_2"  # ✅ 덮어쓰기 방지
project_dir = "C:/content/runs/detect"

# 시간 측정
start_time = time.time()

# 모델 불러오기 및 재학습
model = YOLO(model_path)
results = model.train(
    data= r"C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml",
    epochs=epoch,  # 재학습은 보통 적은 에폭으로
    imgsz=640,
    project=project_dir,
    name=run_name,
    exist_ok=True
)

# 학습 시간 기록
elapsed_minutes = round((time.time() - start_time) / 60, 2)

# 결과 정리
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "model": model_path,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_m/weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_m_fine_2, nbs=64, nms=False, opset=None, opti

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all        253        944      0.994      0.997      0.995      0.994
            100mg         31         31      0.998          1      0.995      0.995
                       7          7      0.991          1      0.995      0.995
        ()          5          5          1          1      0.995      0.995
    ()()         87         87      0.999          1      0.995      0.994
          500/20mg          3          3          1          1      0.995      0.995
              5mg         20         20      0.996          1      0.995      0.989
               10mg          7          7      0.989          1      0.995      0.995
          ()          6          6      0.988          1      0.995      0.995
          10mg/          8          8          1          1      0.995      0.995
          800mg         29         29      0.998          1      0.995      0.995
             20mg          6          6      0.991          1      0.995      0.995
         30mg

In [ ]:
import datetime

# 모델 및 경로
base_model_dir = "C:/content/runs/detect/predict_m/weights"
model_path = os.path.join(base_model_dir, "best.pt")
project_dir = "C:/content/runs/detect"

# 새 데이터셋 이름 + 시간 자동
dataset_name = "dataset_v3"
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
model_size = "m"
run_name = f"{dataset_name}_{model_size}_fine_{timestamp}"

# 학습 실행
start_time = time.time()
model = YOLO(model_path)
results = model.train(
    data=r"C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml",
    epochs=epoch,
    imgsz=640,
    project=project_dir,
    name=run_name,
    exist_ok=False   # ✅ 덮어쓰기 방지
)

# 결과 저장
elapsed_minutes = round((time.time() - start_time) / 60, 2)
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]
    output = {
        "model": model_path,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }
    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)
    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


## YOLO11s

In [15]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# 기존 모델 로드 (best.pt)
base_model_dir = "C:/content/runs/detect/predict_s/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 새 프로젝트 설정
model_size = "s"
run_name = f"predict_{model_size}_fine_2"  # ✅ 덮어쓰기 방지
project_dir = "C:/content/runs/detect"

# 시간 측정
start_time = time.time()

# 모델 불러오기 및 재학습
model = YOLO(model_path)
results = model.train(
    data= r"C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml",
    epochs=100,  # 재학습은 보통 적은 에폭으로
    imgsz=640,
    batch=8,
    project=project_dir,
    name=run_name,
    augment=False,
    exist_ok=True
)

# 학습 시간 기록
elapsed_minutes = round((time.time() - start_time) / 60, 2)

# 결과 정리
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "model": model_path,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_s/weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_s_fine_2, nbs=64, nms=False, opset=None, optim

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all        253        944      0.991      0.998      0.995      0.993
            100mg         31         31      0.998          1      0.995      0.995
                       7          7      0.987          1      0.995      0.995
        ()          5          5      0.988          1      0.995      0.995
    ()()         87         87      0.999          1      0.995      0.994
          500/20mg          3          3      0.974          1      0.995      0.995
              5mg         20         20          1          1      0.995       0.99
               10mg          7          7      0.989          1      0.995      0.981
          ()          6          6      0.987          1      0.995      0.995
          10mg/          8          8          1          1      0.995      0.995
          800mg         29         29          1          1      0.995      0.995
             20mg          6          6          1          1      0.995      0.995
         30mg

## YOLO11n

In [30]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# 기존 모델 로드 (best.pt)
base_model_dir = "C:/content/runs/detect/predict_n/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 새 프로젝트 설정
model_size = "n"
run_name = f"predict_{model_size}_fine_2"  # ✅ 덮어쓰기 방지
project_dir = "C:/content/runs/detect"

# 시간 측정
start_time = time.time()

# 모델 불러오기 및 재학습
model = YOLO(model_path)
results = model.train(
    data= r"C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml",
    epochs=100,  # 재학습은 보통 적은 에폭으로
    imgsz = 640,
    project=project_dir,
    name=run_name,
    augment=False,
    exist_ok=True
)

# 학습 시간 기록
elapsed_minutes = round((time.time() - start_time) / 60, 2)

# 결과 정리
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "model": model_path,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_n/weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_n_fine_2, nbs=64, nms=False, opset=None, opti

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all        253        944      0.989      0.997      0.994      0.991
            100mg         31         31      0.997          1      0.995      0.995
                       7          7      0.993          1      0.995      0.995
        ()          5          5      0.989          1      0.995      0.995
    ()()         87         87      0.999          1      0.995      0.993
          500/20mg          3          3      0.984          1      0.995      0.995
              5mg         20         20      0.996          1      0.995      0.985
               10mg          7          7      0.994          1      0.995      0.995
          ()          6          6      0.991          1      0.995      0.995
          10mg/          8          8      0.993          1      0.995      0.977
          800mg         29         29      0.998          1      0.995      0.995
             20mg          6          6      0.988          1      0.995      0.995
         30mg

# json 파일 저장

In [17]:
import os
import re
import json
import time
import math
import pandas as pd
from glob import glob
from PIL import Image
from ultralytics import YOLO

# =========================
# 0) 경로 설정 (필요에 맞게 수정)
# =========================
# 학습된 모델(best.pt)
best_model_path = r"C:\content\runs\detect\predict_s\weights\best.pt"

# 테스트 이미지 폴더
test_dir = r"C:\Users\kdh12\python\test_images"

# 예측 산출물 폴더
save_dir = r"C:\Users\kdh12\python\dataset_v2\predictions"
run_name = "test_pred"
os.makedirs(save_dir, exist_ok=True)

# (중요) COCO train json: cls index → 실제 category_id 매핑용
#   - 당신이 YOLO 데이터셋 만든 출발점 COCO JSON(훈련용)을 지정하세요.
#   - categories 배열의 순서가 YOLO 클래스 인덱스와 동일해야 합니다.
train_coco_json = r"C:\Users\kdh12\python\dataset_v2\merged_coco_final_train.json"

# 최종 제출 CSV 경로
output_csv = os.path.join(save_dir, "submission_target.csv")

# =========================
# 1) 모델 로드 & 예측 수행
# =========================
model = YOLO(best_model_path)

start_time = time.time()
results = model.predict(
    source=test_dir,
    save_txt=True,     # labels/*.txt 저장
    save_conf=True,    # conf 포함
    project=save_dir,
    name=run_name,
    exist_ok=True,
    imgsz=640
)
elapsed = time.time() - start_time
print(f"✅ Prediction finished in {elapsed/60:.2f} min")

# =========================
# 2) cls index → COCO category_id 매핑표 만들기
# =========================
with open(train_coco_json, "r", encoding="utf-8") as f:
    coco = json.load(f)

# categories 배열의 순서가 YOLO 학습 시 사용된 names 순서와 같다고 가정
# 예: {0: categories[0]['id'], 1: categories[1]['id'], ...}
cls_to_catid = {i: cat["id"] for i, cat in enumerate(coco["categories"])}

# =========================
# 3) 이미지 크기 캐시 (픽셀 변환용)
# =========================
# YOLO txt는 (xc, yc, w, h) 정규화 값 → 픽셀 변환하려면 (W,H) 필요
def get_image_size(img_path):
    with Image.open(img_path) as im:
        return im.size  # (W, H)

# 이미지 파일명 → (W,H) 캐시
size_cache = {}

# =========================
# 4) YOLO txt → 타깃 CSV 변환
# =========================
labels_dir = os.path.join(save_dir, run_name, "labels")
txt_paths = sorted(glob(os.path.join(labels_dir, "*.txt")))

rows = []
annotation_id = 1

for txt_path in txt_paths:
    file_stem = os.path.basename(txt_path)[:-4]  # e.g., 'image_001'
    # image_id: 파일명에서 숫자 추출 (없으면 순번)
    m = re.findall(r"\d+", file_stem)
    image_id = int(m[0]) if m else annotation_id

    # 원본 이미지 경로 추정 (jpg/png 둘 다 탐색)
    candidates = [
        os.path.join(test_dir, file_stem + ext)
        for ext in [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"]
    ]
    img_path = next((p for p in candidates if os.path.exists(p)), None)
    if img_path is None:
        # 이미지 못 찾으면 스킵
        print(f"⚠️ Image not found for {file_stem}, skipping.")
        continue

    if img_path not in size_cache:
        size_cache[img_path] = get_image_size(img_path)
    W, H = size_cache[img_path]

    with open(txt_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 6:
                continue
            cls = int(parts[0])
            xc, yc, w, h, conf = map(float, parts[1:6])

            # (xc,yc,w,h) [0~1] → 좌상단 (x,y), 가로세로 (w,h) [픽셀]
            px_w = w * W
            px_h = h * H
            px_x = (xc * W) - (px_w / 2.0)
            px_y = (yc * H) - (px_h / 2.0)

            # 음수/경계값 보정 + 정수화
            x0 = max(0, round(px_x))
            y0 = max(0, round(px_y))
            ww = max(0, round(px_w))
            hh = max(0, round(px_h))

            # cls index → 실제 category_id
            category_id = cls_to_catid.get(cls, cls)  # 없으면 원래 cls를 fallback

            rows.append({
                "annotation_id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox_x": x0,
                "bbox_y": y0,
                "bbox_w": ww,
                "bbox_h": hh,
                "score": conf
            })
            annotation_id += 1

# =========================
# 5) CSV 저장 (컬럼/순서 고정)
# =========================
cols = ["annotation_id", "image_id", "category_id",
        "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"]
df = pd.DataFrame(rows, columns=cols)
df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✅ Saved CSV: {output_csv}")
print(df.head())



image 1/843 C:\Users\kdh12\python\test_images\1.png: 640x512 1  10mg, 1  800mg, 1  5mg, 1 , 28.8ms
image 2/843 C:\Users\kdh12\python\test_images\10.png: 640x512 1  100mg, 1  20mg, 1 , 1  5mg, 11.7ms
image 3/843 C:\Users\kdh12\python\test_images\100.png: 640x512 1  100mg, 1  30mg, 1  5mg, 1 , 10.5ms
image 4/843 C:\Users\kdh12\python\test_images\1003.png: 640x512 1 ()(), 1  20mg, 1  50/1000mg, 1  40/5mg, 11.4ms
image 5/843 C:\Users\kdh12\python\test_images\1004.png: 640x512 1 ()(), 1  20mg, 1  50/1000mg, 1  40/5mg, 9.0ms
image 6/843 C:\Users\kdh12\python\test_images\1005.png: 640x512 1 ()(), 1  20mg, 1  50/1000mg, 1  40/5mg, 12.3ms
image 7/843 C:\Users\kdh12\python\test_images\1006.png: 640x512 1 ()(), 1  5/100mg, 1  50/850mg, 1  75mg, 12.1ms
image 8/843 C:\Users\kdh12\python\test_images\1007.png: 640x512 1 ()(), 1  5/100mg, 1  50/850mg, 1  75mg, 10.5ms
image 9/843 C:\Users\kdh12\python\test_images\1009.png: 640x512 1 ()(), 1  5/100mg, 1  50mg, 1  20mg, 9.4ms
image 10/843 C:\Users\kdh12

# 3차 도전

## m

In [32]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# ==============================
# ⚙️ 1️⃣ 기본 설정
# ==============================
# 기존 모델 경로
base_model_dir = "C:/content/runs/detect/predict_m/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 이름
dataset_name = "dataset_v2"

# YOLO 프로젝트 루트
project_dir = "C:/content/runs/detect"

# 모델 크기 (s, n, m, l 등)
model_size = "m"

# ✅ run_name은 데이터셋 이름 기반으로만 지정
run_name = f"predict_{model_size}_{dataset_name}"

# ==============================
# 🧠 2️⃣ 학습 시작
# ==============================
start_time = time.time()
print(f"🚀 Now training: {run_name}")

model = YOLO(model_path)
results = model.train(
    data=r"C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml",
    epochs=100,          # 🔧 필요 시 조정
    imgsz=640,           # 이미지 크기
    batch=16,            # GPU 성능에 맞게 (auto, 8, 16, 32 등)
    project=project_dir, # 결과 저장 루트
    name=run_name,       # 폴더 이름 = predict_m_dataset_v3
    exist_ok=False       # ✅ 덮어쓰기 방지 (_2, _3 등 자동 생성)
)

# ==============================
# ⏱️ 3️⃣ 학습 시간 기록
# ==============================
elapsed_minutes = round((time.time() - start_time) / 60, 2)
print(f"✅ Training completed in {elapsed_minutes} min")

# ==============================
# 📊 4️⃣ results.csv → results.json 변환
# ==============================
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "model": model_path,
        "dataset": dataset_name,
        "run_name": run_name,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


🚀 Now training: predict_m_dataset_v2
Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_m/weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_m_datas

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all        253        944      0.994      0.997      0.995      0.994
            100mg         31         31      0.998          1      0.995      0.995
                       7          7      0.991          1      0.995      0.995
        ()          5          5          1          1      0.995      0.995
    ()()         87         87      0.999          1      0.995      0.994
          500/20mg          3          3          1          1      0.995      0.995
              5mg         20         20      0.996          1      0.995      0.989
               10mg          7          7      0.989          1      0.995      0.995
          ()          6          6      0.988          1      0.995      0.995
          10mg/          8          8          1          1      0.995      0.995
          800mg         29         29      0.998          1      0.995      0.995
             20mg          6          6      0.991          1      0.995      0.995
         30mg

In [42]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# ==============================
# ⚙️ 1️⃣ 기본 설정
# ==============================
# 🔹 이전에 학습된 best.pt 경로 (기존 모델)
base_model_dir = "C:/content/runs/detect/predict_m_dataset_v2/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 🔹 재학습할 데이터셋 이름만 바꿔주세요
dataset_name = "dataset_v4"

# 🔹 결과 저장 경로 (YOLO가 자동 폴더 생성)
project_dir = "C:/content/runs/detect"
model_size = "m"

# ✅ run_name: "predict_m_dataset_v4_retrain"
run_name = f"predict_{model_size}_{dataset_name}_retrain"

# ==============================
# 🧠 2️⃣ 모델 불러오기 및 재학습
# ==============================
start_time = time.time()
print(f"🚀 Re-training model: {run_name}")

# 기존 모델 불러오기
model = YOLO(model_path)

# 재학습 실행
results = model.train(
    data=r"C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml",  # 새로운 데이터셋
    epochs=50,            # 🔧 재학습이므로 짧게 (ex. 20~100)
    imgsz=512,
    batch=4,
    project=project_dir,  # 저장 루트
    name=run_name,        # 폴더 이름
    exist_ok=False,       # ✅ 덮어쓰기 방지
    resume=False          # ✅ 기존 학습 이어서가 아니라 새로 시작
)

# ==============================
# ⏱️ 3️⃣ 학습 시간 계산
# ==============================
elapsed_minutes = round((time.time() - start_time) / 60, 2)
print(f"✅ Re-training completed in {elapsed_minutes} min")

# ==============================
# 📊 4️⃣ results.csv → results.json 변환
# ==============================
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "base_model": model_path,
        "dataset": dataset_name,
        "run_name": run_name,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


🚀 Re-training model: predict_m_dataset_v4_retrain
Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_m_dataset_v2/weights\best.pt, momentum=0.937, mo

RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 41, in do_one_step
    data = pin_memory(data, device)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 75, in pin_memory
    {k: pin_memory(sample, device) for k, sample in data.items()}
  File "c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 75, in <dictcomp>
    {k: pin_memory(sample, device) for k, sample in data.items()}
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 64, in pin_memory
    return data.pin_memory(device)
           ^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: CUDA error:

## s

In [ ]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# ==============================
# ⚙️ 1️⃣ 기본 설정
# ==============================
# 기존 모델 경로
base_model_dir = "C:/content/runs/detect/predict_m/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 이름
dataset_name = "dataset_v2"

# YOLO 프로젝트 루트
project_dir = "C:/content/runs/detect"

# 모델 크기 (s, n, m, l 등)
model_size = "s"

# ✅ run_name은 데이터셋 이름 기반으로만 지정
run_name = f"predict_{model_size}_{dataset_name}"

# ==============================
# 🧠 2️⃣ 학습 시작
# ==============================
start_time = time.time()
print(f"🚀 Now training: {run_name}")

model = YOLO(model_path)
results = model.train(
    data=r"C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml",
    epochs=100,          # 🔧 필요 시 조정
    imgsz=640,           # 이미지 크기
    project=project_dir, # 결과 저장 루트
    name=run_name,       # 폴더 이름 = predict_m_dataset_v3
    exist_ok=False       # ✅ 덮어쓰기 방지 (_2, _3 등 자동 생성)
)

# ==============================
# ⏱️ 3️⃣ 학습 시간 기록
# ==============================
elapsed_minutes = round((time.time() - start_time) / 60, 2)
print(f"✅ Training completed in {elapsed_minutes} min")

# ==============================
# 📊 4️⃣ results.csv → results.json 변환
# ==============================
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "model": model_path,
        "dataset": dataset_name,
        "run_name": run_name,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


🚀 Now training: predict_s_dataset_v2
Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_m/weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_s_datas

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all        253        944      0.994      0.997      0.995      0.994
            100mg         31         31      0.998          1      0.995      0.995
                       7          7      0.991          1      0.995      0.995
        ()          5          5          1          1      0.995      0.995
    ()()         87         87      0.999          1      0.995      0.994
          500/20mg          3          3          1          1      0.995      0.995
              5mg         20         20      0.996          1      0.995      0.989
               10mg          7          7      0.989          1      0.995      0.995
          ()          6          6      0.988          1      0.995      0.995
          10mg/          8          8          1          1      0.995      0.995
          800mg         29         29      0.998          1      0.995      0.995
             20mg          6          6      0.991          1      0.995      0.995
         30mg

In [ ]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# ==============================
# ⚙️ 1️⃣ 기본 설정
# ==============================
# 🔹 이전에 학습된 best.pt 경로 (기존 모델)
base_model_dir = "C:/content/runs/detect/predict_s_dataset_v2/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 🔹 재학습할 데이터셋 이름만 바꿔주세요
dataset_name = "dataset_v4"

# 🔹 결과 저장 경로 (YOLO가 자동 폴더 생성)
project_dir = "C:/content/runs/detect"
model_size = "s"

# ✅ run_name: "predict_m_dataset_v4_retrain"
run_name = f"predict_{model_size}_{dataset_name}_retrain"

# ==============================
# 🧠 2️⃣ 모델 불러오기 및 재학습
# ==============================
start_time = time.time()
print(f"🚀 Re-training model: {run_name}")

# 기존 모델 불러오기
model = YOLO(model_path)

# 재학습 실행
results = model.train(
    data=r"C:\Users\kdh12\python\drive-download-20251027T010517Z-1-001\yolo_dataset\data.yaml",  # 새로운 데이터셋
    epochs=50,            # 🔧 재학습이므로 짧게 (ex. 20~100)
    imgsz=640,
    batch=16,             # GPU VRAM에 맞게 조정
    project=project_dir,  # 저장 루트
    name=run_name,        # 폴더 이름
    exist_ok=False,       # ✅ 덮어쓰기 방지
    resume=False          # ✅ 기존 학습 이어서가 아니라 새로 시작
)

# ==============================
# ⏱️ 3️⃣ 학습 시간 계산
# ==============================
elapsed_minutes = round((time.time() - start_time) / 60, 2)
print(f"✅ Re-training completed in {elapsed_minutes} min")

# ==============================
# 📊 4️⃣ results.csv → results.json 변환
# ==============================
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "base_model": model_path,
        "dataset": dataset_name,
        "run_name": run_name,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


🚀 Re-training model: predict_s_dataset_v4_retrain
Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\dataset_v4\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_s_dataset_v2/weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=Fals

RuntimeError: Dataset 'C://Users/kdh12/python/dataset_v4/yolo_dataset/data.yaml' error  'C:\Users\kdh12\python\dataset_v4\yolo_dataset\data.yaml' does not exist

## n

In [34]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# ==============================
# ⚙️ 1️⃣ 기본 설정
# ==============================
# 기존 모델 경로
base_model_dir = "C:/content/runs/detect/predict_m/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 이름
dataset_name = "dataset_v2"

# YOLO 프로젝트 루트
project_dir = "C:/content/runs/detect"

# 모델 크기 (s, n, m, l 등)
model_size = "n"

# ✅ run_name은 데이터셋 이름 기반으로만 지정
run_name = f"predict_{model_size}_{dataset_name}"

# ==============================
# 🧠 2️⃣ 학습 시작
# ==============================
start_time = time.time()
print(f"🚀 Now training: {run_name}")

model = YOLO(model_path)
results = model.train(
    data=r"C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml",
    epochs=100,          # 🔧 필요 시 조정
    imgsz=640,           # 이미지 크기
    batch=16,            # GPU 성능에 맞게 (auto, 8, 16, 32 등)
    project=project_dir, # 결과 저장 루트
    name=run_name,       # 폴더 이름 = predict_m_dataset_v3
    exist_ok=False       # ✅ 덮어쓰기 방지 (_2, _3 등 자동 생성)
)

# ==============================
# ⏱️ 3️⃣ 학습 시간 기록
# ==============================
elapsed_minutes = round((time.time() - start_time) / 60, 2)
print(f"✅ Training completed in {elapsed_minutes} min")

# ==============================
# 📊 4️⃣ results.csv → results.json 변환
# ==============================
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "model": model_path,
        "dataset": dataset_name,
        "run_name": run_name,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


🚀 Now training: predict_n_dataset_v2
Ultralytics 8.3.223  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\kdh12\python\dataset_v2\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_m/weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=predict_n_datas

c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\kdh12\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 

                   all        253        944      0.994      0.997      0.995      0.994
            100mg         31         31      0.998          1      0.995      0.995
                       7          7      0.991          1      0.995      0.995
        ()          5          5          1          1      0.995      0.995
    ()()         87         87      0.999          1      0.995      0.994
          500/20mg          3          3          1          1      0.995      0.995
              5mg         20         20      0.996          1      0.995      0.989
               10mg          7          7      0.989          1      0.995      0.995
          ()          6          6      0.988          1      0.995      0.995
          10mg/          8          8          1          1      0.995      0.995
          800mg         29         29      0.998          1      0.995      0.995
             20mg          6          6      0.991          1      0.995      0.995
         30mg

In [ ]:
import os
import time
import json
import pandas as pd
from ultralytics import YOLO

# ==============================
# ⚙️ 1️⃣ 기본 설정
# ==============================
# 🔹 이전에 학습된 best.pt 경로 (기존 모델)
base_model_dir = "C:/content/runs/detect/predict_n_dataset_v2/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 🔹 재학습할 데이터셋 이름만 바꿔주세요
dataset_name = "dataset_v4"

# 🔹 결과 저장 경로 (YOLO가 자동 폴더 생성)
project_dir = "C:/content/runs/detect"
model_size = "n"

# ✅ run_name: "predict_m_dataset_v4_retrain"
run_name = f"predict_{model_size}_{dataset_name}_retrain"

# ==============================
# 🧠 2️⃣ 모델 불러오기 및 재학습
# ==============================
start_time = time.time()
print(f"🚀 Re-training model: {run_name}")

# 기존 모델 불러오기
model = YOLO(model_path)

# 재학습 실행
results = model.train(
    data=rf"C:\Users\kdh12\python\{dataset_name}\yolo_dataset\data.yaml",  # 새로운 데이터셋
    epochs=50,            # 🔧 재학습이므로 짧게 (ex. 20~100)
    imgsz=640,
    batch=16,             # GPU VRAM에 맞게 조정
    project=project_dir,  # 저장 루트
    name=run_name,        # 폴더 이름
    exist_ok=False,       # ✅ 덮어쓰기 방지
    resume=False          # ✅ 기존 학습 이어서가 아니라 새로 시작
)

# ==============================
# ⏱️ 3️⃣ 학습 시간 계산
# ==============================
elapsed_minutes = round((time.time() - start_time) / 60, 2)
print(f"✅ Re-training completed in {elapsed_minutes} min")

# ==============================
# 📊 4️⃣ results.csv → results.json 변환
# ==============================
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "base_model": model_path,
        "dataset": dataset_name,
        "run_name": run_name,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")
